In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Name, real_name, overall_Score, history_text, intelligence_score, strength_score, speed_score,
# durability_score, power_Score, combat_Score, superpowers, place_of_birth, occupation, gender,
# type_race, height, weight

# powers_text, alter_egos, alignment, skin_color

import pandas as pd
df = pd.read_csv('/content/drive/My Drive/MS_Colabs/superheroes_nlp_dataset (1).csv')
display(df)

<ipython-input-2-05c937e5cf43>:8: DtypeWarning: Columns (225,226,227,228,229,230,248,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,...,Unnamed: 1258,Unnamed: 1259,Unnamed: 1260,Unnamed: 1261,Unnamed: 1262,Unnamed: 1263,Unnamed: 1264,Unnamed: 1265,Unnamed: 1266,Unnamed: 1267
0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,NaN,100,20,30,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1466,Zero,DWN-∞: Zero,DWN-∞: Zero,18,Zero was created by the late Dr. Albert Wily ...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1467,rol all Reploids worldwide with the Mother Elf,a massive armor was constructed around Omega ...,Omega received new weapons,suitable for his new size and weight. Unknown...,which was absolutely loyal to Dr. Weil's orde...,the baby elves would be redundant as they wer...,Zero,in possession of a new body Ciel's ancestor c...,would wake up so he could fight alongside X a...,which were now known as the Elf Wars,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1468,Zoom (New 52),Hunter Zolomon,NaN,20,"Hunter Zolomon is better known as Zoom, a spee...",After tricking Barry Allen and Wally West into...,95,50,100,75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1469,Zoom,Hunter Zolomon,Hunter Zolomon,9,Hunter Zolomon had a troubled relationship wi...,"Zoom is able to alter time, to make himself ev...",75,10,100,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df_name = df[['name']]
df_name = df_name.dropna()
name_list = list(df_name['name'])

sentences = name_list


import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Dense, Lambda, Layer
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.losses import binary_crossentropy

# # Sample data
# sentences = [
#     "This is a sample sentence 1.",
#     "Another example sentence 2.",
#     "Text data for VAE 3.",
#     # Add your 1397 sentences here
# ]

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
total_words = len(tokenizer.word_index) + 1

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(sentences)

# Pad sequences to have consistent length
max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')

# Define the VAE model
latent_dim = 10  # Adjust as needed

# Encoder
inputs = Input(shape=(max_sequence_length,))
h = Dense(256, activation='relu')(inputs)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

# Sampling function
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

# Use Lambda layer to include the sampling function in the model
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# Decoder
decoder_h = Dense(256, activation='relu')
decoder_mean = Dense(total_words, activation='softmax')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

# Custom loss layer
class CustomVariationalLayer(Layer):
    def vae_loss(self, x, x_decoded_mean, z_mean, z_log_var):
        xent_loss = binary_crossentropy(x, x_decoded_mean)
        kl_loss = -0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return xent_loss + kl_loss

    def call(self, inputs):
        x = inputs[0]
        x_decoded_mean = inputs[1]
        z_mean = inputs[2]
        z_log_var = inputs[3]
        loss = self.vae_loss(x, x_decoded_mean, z_mean, z_log_var)
        self.add_loss(loss, inputs=inputs)
        return x

# Instantiate the VAE model
y = CustomVariationalLayer()([inputs, x_decoded_mean, z_mean, z_log_var])
vae = Model(inputs, y)

# Compile the model
vae.compile(optimizer='rmsprop', loss=None)

# Train the model
vae.fit(padded_sequences, epochs=50, batch_size=32)

# Generate sentences using the trained model
def generate_sentences(model, tokenizer, num_sentences=10):
    generated_sentences = []

    for _ in range(num_sentences):
        # Generate random noise for the latent vector
        random_latent_vector = np.random.normal(size=(1, latent_dim))
        # Decode the latent vector to generate a new sentence
        generated_sequence = model.layers[-5].predict(random_latent_vector)
        # Convert sequence back to text
        generated_text = tokenizer.sequences_to_texts([generated_sequence])[0]
        generated_sentences.append(generated_text)

    return generated_sentences

# Generate and print sentences
generated_sentences = generate_sentences(vae, tokenizer)
for i, sentence in enumerate(generated_sentences, start=1):
    print(f"Generated Sentence {i}: {sentence}")


ValueError: ignored